In [55]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as plt
import sys
import os
sys.path.append(os.path.abspath(".."))
from utils import create_geometry_column, find_similar_phrases

# Data load

In [2]:
# Loading data
df = pd.read_csv("../data/nypd-motor-vehicle-collisions.csv", low_memory=False)

# Dataset information

In [3]:
print(df.shape)

(1612178, 29)


In [4]:
print(df.columns)

Index(['ACCIDENT DATE', 'ACCIDENT TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')


In [5]:
df.head()

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30T00:00:00.000,21:53,BRONX,10460,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612178 entries, 0 to 1612177
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   ACCIDENT DATE                  1612178 non-null  object 
 1   ACCIDENT TIME                  1612178 non-null  object 
 2   BOROUGH                        1127553 non-null  object 
 3   ZIP CODE                       1127376 non-null  object 
 4   LATITUDE                       1415893 non-null  float64
 5   LONGITUDE                      1415893 non-null  float64
 6   LOCATION                       1415893 non-null  object 
 7   ON STREET NAME                 1298002 non-null  object 
 8   CROSS STREET NAME              1079193 non-null  object 
 9   OFF STREET NAME                219732 non-null   object 
 10  NUMBER OF PERSONS INJURED      1612161 non-null  float64
 11  NUMBER OF PERSONS KILLED       1612145 non-null  float64
 12  NUMBER OF PEDE

In [7]:
df.describe()

,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
count,1.415893e+06,1.415893e+06,1.612161e+06,1.612145e+06,1.612178e+06,1.612178e+06,1.612178e+06,1.612178e+06,1.612178e+06,1.612178e+06,1.612178e+06
mean,4.068864e+01,-7.386657e+01,2.631363e-01,1.185998e-03,5.060483e-02,6.302034e-04,2.098590e-02,9.242156e-05,1.916854e-01,4.633483e-04,2.765946e+06
std,1.200403e+00,2.438640e+00,6.584832e-01,3.644859e-02,2.316972e-01,2.577872e-02,1.445222e-01,9.677481e-03,6.206497e-01,2.334547e-02,1.506373e+06
min,0.000000e+00,-2.012371e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.300000e+01
25%,4.066882e+01,-7.397746e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.014464e+06
50%,4.072258e+01,-7.393002e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.422826e+06
75%,4.076789e+01,-7.386727e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.854210e+06
max,4.231832e+01,0.000000e+00,3.100000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,3.100000e+01,5.000000e+00,4.249104e+06


# Data Cleaning

## Deleting unnecessary columns

In [8]:
del df["ZIP CODE"]

## Datetime

In [9]:
df["ACCIDENT DATE"]

0          2019-08-05T00:00:00.000
1          2019-08-27T00:00:00.000
2          2019-08-15T00:00:00.000
3          2019-08-30T00:00:00.000
4          2019-08-06T00:00:00.000
                    ...           
1612173    2012-07-21T00:00:00.000
1612174    2012-07-09T00:00:00.000
1612175    2012-07-09T00:00:00.000
1612176    2012-07-18T00:00:00.000
1612177    2012-07-12T00:00:00.000
Name: ACCIDENT DATE, Length: 1612178, dtype: object

In [10]:
df["ACCIDENT DATE"] = pd.to_datetime(df["ACCIDENT DATE"], format="%Y-%m-%dT%H:%M:%S.%f")
df["ACCIDENT DATE FORMATTED"] = df["ACCIDENT DATE"].dt.strftime("%d/%m/%Y")
df["ACCIDENT DATE"] = df["ACCIDENT DATE"].dt.normalize()

In [11]:
df["ACCIDENT DATE"].head(5)

0   2019-08-05
1   2019-08-27
2   2019-08-15
3   2019-08-30
4   2019-08-06
Name: ACCIDENT DATE, dtype: datetime64[ns]

In [13]:
df["ACCIDENT TIME"].head(5)

0          16:30
1          16:02
2          17:57
3          21:53
4           9:45
           ...  
1612173    22:46
1612174    16:45
1612175    14:35
1612176     9:20
1612177     3:10
Name: ACCIDENT TIME, Length: 1612178, dtype: object

## Missing boroughs, latitude, longitude and location

In [13]:
df["BOROUGH"].unique()

array(['QUEENS', 'BROOKLYN', 'MANHATTAN', 'BRONX', nan, 'STATEN ISLAND'],
      dtype=object)

In [14]:
# Check how many boroughs are missing
missing_boroughs = df["BOROUGH"].isna().mean()
print(f"Missing boroughs {round(missing_boroughs*100,2)} %")

Missing boroughs 30.06 %


In [15]:
df[["LATITUDE", "LONGITUDE"]].isna().sum()

LATITUDE     196285
LONGITUDE    196285
dtype: int64

In [20]:
# Check if both - latitude and longitude is missing in observation
missing_count = df[df["LATITUDE"].isna() & df["LONGITUDE"].isna()].shape[0]
print(f"Missing both latitude and longitude: {missing_count}")

Missing both latitude and longitude: 196285


## Match coordinates with boroughs from geojson

### Create geometry df

In [133]:
geometry_df = df.dropna(subset=["LONGITUDE", "LATITUDE"]).copy()
geometry_df = geometry_df[["LONGITUDE", "LATITUDE", "BOROUGH"]]
# create a column with original index as the original one will be replaced after sjoin
geometry_df["ORIGINAL INDEX"] = geometry_df.index
geometry_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1415893 entries, 0 to 1612177
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   LONGITUDE       1415893 non-null  float64
 1   LATITUDE        1415893 non-null  float64
 2   BOROUGH         1099516 non-null  object 
 3   ORIGINAL INDEX  1415893 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 54.0+ MB


### Create geometry column

In [134]:
create_geometry_column(geometry_df)
geometry_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1415893 entries, 0 to 1612177
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   LONGITUDE       1415893 non-null  float64
 1   LATITUDE        1415893 non-null  float64
 2   BOROUGH         1099516 non-null  object 
 3   ORIGINAL INDEX  1415893 non-null  int64  
 4   GEOMETRY        1415893 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 64.8+ MB


### Convert geometry_df into GeoDataFrame

In [105]:
geometry_gdf = gpd.GeoDataFrame(geometry_df, geometry="GEOMETRY", crs="EPSG:4326")

### Load boroughs from geojson

In [106]:
# Load borough boundaries
# https://www.nyc.gov/content/planning/pages/resources/datasets/borough-boundaries
boroughs_gdf = gpd.read_file("../data/nybb.shp")

In [107]:
print(boroughs_gdf.columns)
print(boroughs_gdf.crs)

Index(['BoroCode', 'BoroName', 'Shape_Leng', 'Shape_Area', 'geometry'], dtype='object')
EPSG:2263


In [108]:
boroughs_gdf = boroughs_gdf.to_crs(epsg=4326)

### Spatial join

In [109]:
print(boroughs_gdf.columns)

Index(['BoroCode', 'BoroName', 'Shape_Leng', 'Shape_Area', 'geometry'], dtype='object')


In [123]:
geometry_gdf_joined = gpd.sjoin(geometry_gdf, boroughs_gdf[["BoroName", "geometry"]], how = "left", predicate = "intersects")

In [124]:
print(geometry_gdf_joined["BoroName"].unique())

['Queens' 'Brooklyn' 'Manhattan' 'Bronx' 'Staten Island' nan]


In [125]:
geometry_gdf_joined["BoroName"].isna().sum()

np.int64(9256)

In [126]:
geometry_gdf_joined["BoroName"] = geometry_gdf_joined["BoroName"].fillna("Unknown")

In [127]:
geometry_gdf_joined["BoroName"].unique()

array(['Queens', 'Brooklyn', 'Manhattan', 'Bronx', 'Staten Island',
       'Unknown'], dtype=object)

In [128]:
geometry_gdf_joined["BoroName"] = geometry_gdf_joined["BoroName"].astype(str).str.upper()

In [132]:
geometry_gdf_joined.head(5)

,LONGITUDE,LATITUDE,BOROUGH,ORIGINAL INDEX,GEOMETRY,index_right,BoroName
0,-73.790184,40.676052,QUEENS,0,POINT (-73.79018 40.67605),2.0,QUEENS
1,-73.951096,40.657780,BROOKLYN,1,POINT (-73.9511 40.65778),4.0,BROOKLYN
2,-73.993835,40.718143,MANHATTAN,2,POINT (-73.99384 40.71814),3.0,MANHATTAN
3,-73.866610,40.840534,BRONX,3,POINT (-73.86661 40.84053),1.0,BRONX
4,-73.975400,40.745440,MANHATTAN,4,POINT (-73.9754 40.74544),3.0,MANHATTAN


### Fill NaN with mapped boroughs

In [158]:
df.loc[geometry_gdf_joined["ORIGINAL INDEX"], "BOROUGH"] = df.loc[
    geometry_gdf_joined["ORIGINAL INDEX"], "BOROUGH"
    ].combine_first(geometry_gdf_joined.set_index("ORIGINAL INDEX")["BoroName"])

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612178 entries, 0 to 1612177
Data columns (total 30 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   ACCIDENT DATE                  1612178 non-null  datetime64[ns]
 1   ACCIDENT TIME                  1612178 non-null  object        
 2   BOROUGH                        1443930 non-null  object        
 3   LATITUDE                       1415893 non-null  float64       
 4   LONGITUDE                      1415893 non-null  float64       
 5   LOCATION                       1415893 non-null  object        
 6   ON STREET NAME                 1298002 non-null  object        
 7   CROSS STREET NAME              1079193 non-null  object        
 8   OFF STREET NAME                219732 non-null   object        
 9   NUMBER OF PERSONS INJURED      1612161 non-null  float64       
 10  NUMBER OF PERSONS KILLED       1612145 non-null  float

In [162]:
df['BOROUGH'].unique()

array(['QUEENS', 'BROOKLYN', 'MANHATTAN', 'BRONX', 'STATEN ISLAND', nan,
       'UNKNOWN'], dtype=object)

## Label NaN as "Unknown"

In [163]:
cols_to_fill = ["BOROUGH","LATITUDE","LONGITUDE","LOCATION"]
df[cols_to_fill] = df[cols_to_fill].apply(lambda col: col.fillna("UNKNOWN"))

In [164]:
df['BOROUGH'].unique()

array(['QUEENS', 'BROOKLYN', 'MANHATTAN', 'BRONX', 'STATEN ISLAND',
       'UNKNOWN'], dtype=object)

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612178 entries, 0 to 1612177
Data columns (total 30 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   ACCIDENT DATE                  1612178 non-null  datetime64[ns]
 1   ACCIDENT TIME                  1612178 non-null  object        
 2   BOROUGH                        1612178 non-null  object        
 3   LATITUDE                       1612178 non-null  object        
 4   LONGITUDE                      1612178 non-null  object        
 5   LOCATION                       1612178 non-null  object        
 6   ON STREET NAME                 1298002 non-null  object        
 7   CROSS STREET NAME              1079193 non-null  object        
 8   OFF STREET NAME                219732 non-null   object        
 9   NUMBER OF PERSONS INJURED      1612161 non-null  float64       
 10  NUMBER OF PERSONS KILLED       1612145 non-null  float

## Number of persons

In [19]:
df["NUMBER OF PERSONS INJURED"].unique()

array([ 0.,  1.,  3.,  2.,  4.,  7.,  5.,  6.,  8.,  9., nan, 11., 27.,
       13., 10., 12., 17., 14., 15., 16., 20., 22., 31., 19., 18., 24.])

In [20]:
df["NUMBER OF PERSONS INJURED"] = df["NUMBER OF PERSONS INJURED"].astype("Int64")

In [21]:
df["NUMBER OF PERSONS INJURED"].unique()

<IntegerArray>
[   0,    1,    3,    2,    4,    7,    5,    6,    8,    9, <NA>,   11,   27,
   13,   10,   12,   17,   14,   15,   16,   20,   22,   31,   19,   18,   24]
Length: 26, dtype: Int64

In [22]:
df["NUMBER OF PERSONS KILLED"].unique()

array([ 0.,  1.,  2., nan,  8.,  4.,  5.,  3.])

In [23]:
df["NUMBER OF PERSONS KILLED"] = df["NUMBER OF PERSONS KILLED"].astype("Int64")

In [24]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1612178 entries, 0 to 1612177
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   ACCIDENT DATE                  1612178 non-null  datetime64[ns]
 1   ACCIDENT TIME                  1612178 non-null  object        
 2   BOROUGH                        1612178 non-null  object        
 3   LATITUDE                       1612178 non-null  object        
 4   LONGITUDE                      1612178 non-null  object        
 5   LOCATION                       1612178 non-null  object        
 6   ON STREET NAME                 1298002 non-null  object        
 7   CROSS STREET NAME              1079193 non-null  object        
 8   OFF STREET NAME                219732 non-null   object        
 9   NUMBER OF PERSONS INJURED      1612161 non-null  Int64         
 10  NUMBER OF PERSONS KILLED       1612145 non-null  Int64

In [25]:
df["NUMBER OF MOTORIST KILLED"].unique()

array([0, 1, 2, 5, 3, 4])

## String formatting

In [26]:
cols_to_format = ["CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2", "CONTRIBUTING FACTOR VEHICLE 3",
                     "CONTRIBUTING FACTOR VEHICLE 4", "CONTRIBUTING FACTOR VEHICLE 5", "VEHICLE TYPE CODE 1", 
                     "VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5",
                 "ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"]
df[cols_to_format] = df[cols_to_format].astype(str).apply(
    lambda col: col.str.strip().str.lower())

In [59]:
cfv_typos_cols_1 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 1"])
cfv_typos_cols_2 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 2"])
cfv_typos_cols_3 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 3"])
cfv_typos_cols_4 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 4"])
cfv_typos_cols_5 = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 5"])

In [66]:
print(f"CFV 1 typos {cfv_typos_cols_1}")
print(f"CFV 2 typos {cfv_typos_cols_2}")
print(f"CFV 3 typos {cfv_typos_cols_3}")
print(f"CFV 4 typos {cfv_typos_cols_4}")
print(f"CFV 5 typos {cfv_typos_cols_5}")

CFV 1 typos {'reaction to uninvolved vehicle': ['reaction to other uninvolved vehicle'], 'illnes': ['illness'], 'reaction to other uninvolved vehicle': ['reaction to uninvolved vehicle'], 'illness': ['illnes']}
CFV 2 typos {'reaction to uninvolved vehicle': ['reaction to other uninvolved vehicle'], 'illnes': ['illness'], 'reaction to other uninvolved vehicle': ['reaction to uninvolved vehicle'], 'illness': ['illnes']}
CFV 3 typos {'reaction to uninvolved vehicle': ['reaction to other uninvolved vehicle'], 'illnes': ['illness'], 'illness': ['illnes'], 'reaction to other uninvolved vehicle': ['reaction to uninvolved vehicle']}
CFV 4 typos {}
CFV 5 typos {}


### Corrections directory

In [68]:
corrections_cfv = {"reaction to other uninvolved vehicle":"reaction to uninvolved vehicle", "illnes":"illness"}

In [69]:
cfv_cols_to_correct = ["CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2", "CONTRIBUTING FACTOR VEHICLE 3",
                     "CONTRIBUTING FACTOR VEHICLE 4", "CONTRIBUTING FACTOR VEHICLE 5"]
df[cfv_cols_to_correct] = df[cfv_cols_to_correct].replace(corrections_cfv)

In [70]:
cfv_typos_cols_1_cleaned = find_similar_phrases(df["CONTRIBUTING FACTOR VEHICLE 1"])
print(cfv_typos_cols_1_cleaned)

{}


In [ ]:
# check odd values